# 作业八

一种用于去除**完全重复内容**的简单方法是：只保留在整个语料库中**唯一出现**的行。事实证明，这足以消除很大一部分冗余内容，例如前面提到的页眉和菜单选项。在更简单的情况下，当我们删除在其他地方被**完全重复**的行时，通常可以得到每个页面的**唯一主体内容**（例如 StackOverflow 上的问题和回答）。

为此，我们可以对语料库进行**一次遍历**，统计每一行出现的次数。然后，在**第二次遍历**中，通过只保留其唯一行来重写每个文档。

一种朴素的做法是，使用一个数据结构来保存计数器，但这样会占用与存储语料库中所有唯一行相同规模的内存。一个简单的**内存优化技巧**是：不直接使用整行文本作为键，而是使用该行的**哈希值**作为键，从而使键的大小固定（而不是依赖于行的长度）。现在你将实现这种简单的去重方法。



### 问题（exact_deduplication）：3 分

编写一个函数，接收一组输入文件路径，并对这些文件执行**精确行去重**。该函数应首先统计语料库中每一行的出现频率，并使用**哈希**来降低内存使用。

然后，通过**只保留唯一行**来重写每个文件。

**交付物（Deliverable）：**
一个执行**精确行去重**的函数。你的函数应接收两个参数：
(a) 输入文件路径列表；
(b) 一个输出目录。

该函数应将每个输入文件重写到输出目录中，**文件名保持不变**，但通过删除在输入文件集合中**出现超过一次**的行来实现内容去重。

例如，如果输入路径是 `a/1.txt` 和 `a/2.txt`，输出目录是 `b/`，那么你的函数应生成文件 `b/1.txt` 和 `b/2.txt`。

请实现适配器 **[run_exact_line_deduplication]**，并确保它能够通过以下测试：

```
uv run pytest -k test_exact_line_deduplication
```

In [ ]:
import hashlib
from collections import defaultdict
from typing import List


def _line_hash(line: str) -> str:
    """
    对一行文本计算稳定哈希值
    """
    return hashlib.md5(line.encode("utf-8")).hexdigest()


def exact_deduplication(file_paths: List[str]) -> None:
    """
    对给定文件列表执行精确行去重：
    - 第一次遍历：统计每一行（基于哈希）的全局出现次数
    - 第二次遍历：仅保留全语料中唯一出现的行，重写文件

    参数：
        file_paths: 输入文件路径列表
    """

    # ---------- 第一次遍历：统计 ----------
    hash_counter = defaultdict(int)

    for path in file_paths:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                h = _line_hash(line)
                hash_counter[h] += 1

    # ---------- 第二次遍历：重写 ----------
    for path in file_paths:
        unique_lines = []

        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                h = _line_hash(line)
                if hash_counter[h] == 1:
                    unique_lines.append(line)

        # 覆盖写回文件
        with open(path, "w", encoding="utf-8") as f:
            f.writelines(unique_lines)


# 作业九

### 3.2 MinHash + LSH 文档去重

**精确去重（Exact deduplication）** 对于移除在多个网页中逐字重复的内容非常有用，但它无法处理文档内容仅有轻微差异的情况。
例如，考虑软件许可证文档：许可证通常由一个模板生成，其中只需要填写年份和软件作者的名字。因此，一个 MIT 许可证项目中的 license 文件与另一个 MIT 许可证项目中的 license 文件在内容上高度相似，但它们并不是完全重复的文档。为了去除这类“高度重复但不完全相同”的模板化内容，我们需要**模糊去重（fuzzy deduplication）**。
为了高效地执行文档级别的模糊去重，我们将使用 **MinHash 结合局部敏感哈希（LSH）**。

为了执行模糊去重，我们采用一种特定的文档相似性度量方式：**文档 n-gram 集合之间的 Jaccard 相似度**。集合 $S$ 与 $T$ 的 Jaccard 相似度定义为

$$
\frac{|S \cap T|}{|S \cup T|}
$$

一种朴素的模糊去重方法是：将每个文档表示为一个 n-gram 集合，计算所有文档对之间的 Jaccard 相似度，如果相似度超过某个阈值，就将该文档对标记为重复。然而，这种方法在大规模文档集合（例如 Common Crawl）上是不可行的。此外，直接存储 n-gram 集合在内存上也会比存储原始文档占用更多空间。

---

### MinHash

为了解决内存问题，我们用**签名（signature）**来替代基于 n-gram 集合的文档表示。具体来说，我们希望构造这样一种签名：当我们比较两个文档的签名时，可以得到对原始 n-gram 集合 Jaccard 相似度的近似值。MinHash 签名正好满足这一性质。

为了计算一个文档 n-gram 集合

$$
S = {s_1, \ldots, s_n}
$$

的 MinHash 签名，我们需要 $k$ 个不同的哈希函数 $h_1, \ldots, h_k$。每个哈希函数将一个 n-gram 映射为一个整数。
给定哈希函数 $h_i$，文档 n-gram 集合 $S$ 的 MinHash 值定义为：

$$
\text{minhash}[h_i, S] = \min[h_i]s_1$, h_i$s_2$, \ldots, h_i$s_n$
$$

文档 n-gram 集合 $S$ 的签名是一个位于 $\mathbb{R}^k$ 中的向量，其中每一维对应一个随机哈希函数下的 MinHash 值：
$$
$$\text{minhash}$h_1, S$, \text{minhash}$h_2, S$, \ldots, \text{minhash}$h_k, S$$$
$$

可以证明，对于两个文档的 n-gram 集合 $S_1$ 和 $S_2$，它们的 Jaccard 相似度可以通过 **签名中相同 MinHash 值所占的比例** 来近似（证明见第 3.3 节，参考 *Leskovec et al., 2014*）。
例如，给定两个文档的签名分别为 `[1, 2, 3, 2]` 和 `[5, 2, 3, 4]
`，由于第二列和第三列的 MinHash 值相同，Jaccard 相似度被近似为 $2/4$。

---

### 局部敏感哈希（LSH）

尽管 MinHash 为我们提供了一种**内存高效**且能保持期望相似度的文档表示方式，但我们仍然需要在所有文档对之间进行比较，以找到相似度最高的文档对。
**LSH（Locality-Sensitive Hashing）** 提供了一种高效的方法，可以将**可能具有较高相似度的文档分到同一个桶中**，从而避免全量两两比较。

在将 LSH 应用于文档签名（即 $\mathbb{R}^k$ 中的向量）时，我们会将签名划分为 $b$ 个 band，每个 band 包含 $r$ 个 MinHash 值，其中 $k = b r$。
例如，如果我们有长度为 100 的文档签名（由 100 个随机哈希函数生成），我们可以划分为 **2 个 band，每个 band 含 50 个 minhash**，或者 **4 个 band，每个 band 含 25 个 minhash**，或者 **50 个 band，每个 band 含 2 个 minhash**，等等。
如果两个文档在某一个 band 上具有**完全相同的哈希值**，那么它们就会被聚到同一个桶（bucket）中，并被视为**候选重复文档（candidate duplicates）**。
因此，在签名长度固定的情况下，**增加 band 的数量会提高召回率（recall），但会降低精确率（precision）**。

---



### 具体示例

假设我们有一个文档 $D_1$，其 minhash 签名为
$[1, 2, 3, 4, 5, 6]$，
另一个文档 $D_2$ 的 minhash 签名为
$[1, 2, 3, 5, 1, 2]$。

如果我们使用 **3 个 band，每个 band 含 2 个 minhash**，那么：

* $D_1$ 的第一个 band 是 $[1, 2]$，第二个 band 是 $[3, 4]$，第三个 band 是 $[5, 6]$
* $D_2$ 的第一个 band 是 $[1, 2]$，第二个 band 是 $[3, 5]$，第三个 band 是 $[1, 2]$

由于在**第一个 band** 中的哈希值完全一致（对两个文档都是 $[1, 2]$），$D_1$ 和 $D_2$ 会在该 band 下被聚到同一个桶中。
而在其他 band 中，由于哈希值不匹配，它们不会被聚到同一个桶。

不过需要注意的是：**只要文档在至少一个 band 中被聚到同一个桶里，它们就会被视为候选重复文档**，而不需要其他 band 也匹配。

---

一旦我们识别出了候选重复文档，就可以用多种方式对它们进行后处理。例如，可以计算所有候选文档对之间的 **真实 n-gram Jaccard 相似度**，并将超过某个阈值的文档对标记为重复。

---

最后，我们还需要**跨桶聚类重复文档**。
例如，假设文档 A 和 B 在某一个桶中匹配，并且它们的真实 Jaccard 相似度高于阈值；同时，文档 B 和 C 在另一个桶中匹配，且它们的真实 Jaccard 相似度也高于阈值。
那么，我们会将文档 A、B、C 视为**同一个聚类（cluster）**，并在每个聚类中**随机保留一个文档，其余删除**。

---

## 问题（MinHash 去重）：8 分

编写一个函数，该函数接收一组输入文件路径，并使用 **MinHash + LSH** 执行**模糊文档去重**。具体要求如下：

* 对给定路径列表中的每个文档计算 **MinHash 签名**
* 使用给定数量的 **band** 通过 **LSH** 识别候选重复文档
* 对候选重复文档计算 **真实的 n-gram Jaccard 相似度**
* 删除相似度超过给定阈值的文档

为提升效果（参考 *Penedo et al., 2023*），在计算 MinHash 签名和 / 或比较 Jaccard 相似度之前，应对文本进行如下规范化处理：

* 全部转换为小写
* 移除标点符号
* 规范化空白符
* 移除重音符号（accent）
* 应用 **Unicode NFD 规范化**

---

### 交付物（Deliverable）

实现一个执行模糊文档去重的函数。该函数至少应接收以下参数：

1. 输入文件路径列表
2. 用于计算 MinHash 签名的哈希函数数量
3. LSH 中使用的 band 数量
4. 用于计算 MinHash 签名的 n-gram 长度（以“词”为单位）
5. 输出目录路径

你可以假设：用于计算 MinHash 签名的哈希函数数量可以被 band 数量整除。

---

### 输出要求

你的函数应将每个输入文件写入输出目录，文件名保持不变，但**只写出以下两类文档**：

* $a$ 不是候选重复文档的文档
* $b$ 在聚类后的 bucket 中被**随机选中保留**的文档

例如，如果输入路径是 `a/1.txt` 和 `a/2.txt`，输出目录是 `b/`，那么你的函数应输出 `b/1.txt` 和 `b/2.txt`。

---

请实现适配器函数 **`run_minhash_deduplication`**，并确保它能够通过以下测试：

```bash
pytest -k test_minhash_deduplication
```

---

**注释：**
6. 关于 LSH 和 MinHash 的更深入讨论，请参考 *Leskovec et al., 2014* 第 3 章，在线版本可在 `http://infolab.stanford.edu/~ullman/mmds/ch3.pdf` 获取。
7. 这些哈希函数可以来自同一个哈希函数族，但使用不同的随机种子。例如，MurmurHash3 是一个哈希函数族，使用不同的种子可以实例化出不同的具体哈希函数。




In [1]:
import os
import re
import unicodedata
from collections import defaultdict
from itertools import combinations
import shutil  # 用于复制文件（保留元数据）
from unicodedata import normalize

def normalize_text(text: str) -> str:
    """
    预备工作 1：文本标准化（Normalization）

    目的：
    - 消除与语义无关的差异（大小写、标点、重音、空白）
    - 让“本质相同”的文本在后续 shingle 层面尽可能一致
    - 这是近似去重中“降低噪声”的关键步骤

    如果跳过此步骤：
    - 同一句话可能因为大小写或标点不同而被认为完全不相似
    """

    # 1 全部转为小写
    # WHY：避免 "Apple" 和 "apple" 被当作不同词
    text = text.lower()

    # 2 删除标点符号
    # 正则解释：
    # \w  → 字母、数字、下划线
    # \s  → 空白字符
    # [^\w\s] → 所有“非字母、非数字、非空白”的字符，即标点
    text = re.sub(r'[^\w\s]', '', text)

    # 3 规范化空白字符
    # \s+ 表示一个或多个连续空白
    # 统一替换为一个空格，并去掉首尾空格
    text = re.sub(r'\s+', ' ', text).strip()

    # 4 Unicode NFD 规范化
    # 例如：é → e + ´（把重音符号拆出来）
    text = normalize('NFD', text)

    # 5 删除所有“非间距重音符号”
    # unicodedata.category(ch) == "Mn"
    # Mn = Mark, Nonspacing（重音、变音符）
    text = "".join(
        ch for ch in text
        if unicodedata.category(ch) != "Mn"
    )

    return text

def get_shingles(text: str, n: int) -> list[str]:
    """
    预备工作 2：生成 n-gram（shingles）

    理论背景：
    - MinHash 的输入不是原始文本，而是“集合”
    - 我们用 n-gram 字符子串来近似表示文本内容

    举例：
    text = "hello", n = 3
    → {"hel", "ell", "llo"}

    为什么用 set：
    - Jaccard 相似度基于“集合”，不关心频次
    """

    shingles = set()

    # 滑动窗口提取长度为 n 的子串
    # len(text) - n + 1 确保不会越界
    for i in range(len(text) - n + 1):
        shingles.add(text[i:i+n])

    return shingles


def estimate_jaccard(sig1: list, sig2: list) -> float:
    """
    使用 MinHash 签名估算 Jaccard 相似度

    理论公式：
    J(A, B) ≈ 相等的 MinHash 行数 / 总 hash 数 K

    重要说明：
    - 这是“估算值”，不是精确 Jaccard
    - 但在 K 足够大时，期望值等于真实 Jaccard
    """

    if len(sig1) != len(sig2):
        raise ValueError("Signatures must have the same length.")

    # zip(sig1, sig2)：
    # - 同时遍历两个签名的第 i 行
    # - 判断 hash 是否相等
    matching_hashes = sum(
        1 for h1, h2 in zip(sig1, sig2) if h1 == h2
    )

    return matching_hashes / len(sig1)

def run_minhash_deduplication(
    input_files: list[os.PathLike],
    num_hashes: int,
    num_bands: int,
    ngrams: int,
    jaccard_threshold: float,
    output_directory: os.PathLike,
):
    """
    主函数：使用 MinHash + LSH 完成近似文本去重

    输入：
    - input_files         : 文档路径列表
    - num_hashes (K)      : MinHash 签名长度
    - num_bands (B)       : LSH 分段数量
    - ngrams              : shingle 的 n
    - jaccard_threshold   : 相似度阈值
    - output_directory    : 输出目录
    """
    # 第一步，先读取文件，并进行预备工作处理。
    doc_shingles = defaultdict(set)
    all_shingles = set()# 所有出现过的shingle,也就是所有文档的并集
    for file_path in input_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            normalized_text = normalize_text(text)
            shingles = get_shingles(normalized_text, ngrams)
            doc_shingles[file_path] = shingles
            all_shingles.update(shingles) # 收集所有出现过的shingle
            # print(shingles)
    
    #第二步，生成MinHash签名
    # 初始化签名矩阵 M，所有值为无穷大
    # signatures[doc_id] = [inf, inf, ..., inf]，相当于按列来创建矩阵
    signatures = {doc_id: [float('inf')] * num_hashes for doc_id in doc_shingles}
    # 对每一个唯一的 shingle 计算它的 K 个哈希值
    # 这是按“行r”来计算哈希值 val_1, val_2, ...
    for shingle in all_shingles:
        # 使用不同的 salt (盐)来模拟 k 个独立的哈希函数
        hash_values = [hash(shingle + str(i)) for i in range(num_hashes)]
        # 遍历所有文档，如果文档包含这个 shingle，则更新它的签名
        for doc_id, shingles_set in doc_shingles.items():
            if shingle in shingles_set:
                # 规则：M(i, c) = min(M(i, c), val_i)
                for i in range(num_hashes):
                    signatures[doc_id][i] = min(signatures[doc_id][i], hash_values[i])

    # --- 第三步: LSH分段与分桶 ---
    r = num_hashes // num_bands # 每个band包含的hash签名数量 (rows per band)

    candidate_pairs = set()
    
    # 1. 对每一个band进行处理
    for band_index in range(num_bands):
        # buckets是一个哈希表，用于存放当前band的“哈希桶”
        # key是band的哈希值，value是落入这个桶的文档列表
        buckets = defaultdict(list)
        
        # 2. 遍历所有文档
        for doc_id, sig in signatures.items():
            # 提取当前band对应的签名部分
            start_index = band_index * r
            end_index = start_index + r
            band = tuple(sig[start_index:end_index]) #转为tuple才能作为dict的key
            # 3. 将 (band -> doc_id) 存入桶中
            buckets[band].append(doc_id)
            
        # 4. 生成候选对
        # 只要一个桶里的文档数 > 1，它们就是候选对
        for bucket_docs in buckets.values():
            if len(bucket_docs) > 1:
                # 使用itertools.combinations来生成桶内所有可能的配对
                for pair in combinations(bucket_docs, 2):
                    candidate_pairs.add(tuple(sorted(pair))) #排序后加入set，避免(a,b)和(b,a)重复

    # --- 第四步: 验证候选对并输出结果 ---    
    duplicate_pairs = []
    for doc1, doc2 in candidate_pairs:
        # 从签名矩阵中直接估算Jaccard相似度
        j_estimate = estimate_jaccard(signatures[doc1], signatures[doc2])
        
        if j_estimate >= jaccard_threshold:
            duplicate_pairs.append((doc1, doc2))

    
    # --- 第五步: 根据新要求，构建重复集群并选择要保留的文件 ---
    adj = defaultdict(list)
    for doc1, doc2 in duplicate_pairs:
        adj[doc1].append(doc2)
        adj[doc2].append(doc1)
        
    # 2. 寻找连通分量 (即重复的集群)
    clusters = []
    visited = set()
    for doc in input_files:
        if doc not in visited:
            cluster = []
            q = [doc]
            visited.add(doc)
            head = 0
            while head < len(q):
                current = q[head]
                head += 1
                cluster.append(current)
                # 仅当节点在adj中时才遍历邻居
                if current in adj:
                    for neighbor in adj[current]:
                        if neighbor not in visited:
                            visited.add(neighbor)
                            q.append(neighbor)
            clusters.append(cluster)

    # 3. 决定要保留的文件
    files_to_keep = set()
    for cluster in clusters:
        # 如果一个集群只有一个文件，说明它是唯一的
        if len(cluster) == 1:
            files_to_keep.add(cluster[0])
        else:
            # 如果是重复集群，按字母顺序排序并选择第一个作为代表
            cluster.sort()
            representative = cluster[0]
            files_to_keep.add(representative)


    # --- 第六步: 将选定的文件写入输出目录 ---
    os.makedirs(output_directory, exist_ok=True) # 创建输出目录
    
    copied_count = 0
    for file_path in files_to_keep:
        # 构建目标路径，保持文件名不变
        destination_path = os.path.join(output_directory, os.path.basename(file_path))
        shutil.copy2(file_path, destination_path) # copy2会同时复制元数据
        copied_count += 1    

    return None